In [1]:
%reload_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path
import numpy as np
from scipy import stats
import sys
from datetime import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tueplots import axes, bundles

# Increase the resolution of all the plots below
bundles.beamer_moml()
plt.rcParams.update({"figure.dpi": 200,"figure.facecolor":"w","figure.figsize": (15,10)})

from utilities.utl import remove_outliers_df


## set path

In [21]:
dir_proj = Path("__file__").absolute().parents[1]
dir_raw = Path.joinpath(dir_proj,"04_data","00_raw")
dir_plots = Path.joinpath(dir_proj,"06_plots","02_group_comparisons")
dir_prep = Path.joinpath(dir_proj,"04_data","01_prep")
print(dir_proj)

c:\Users\User\Desktop\kiel\tremor_feedback_jw


## load data
Load the preprocessed data from pupils, force sensor and clinical data

In [25]:
dat_pupil = pd.read_csv(Path.joinpath(dir_prep,"all_trials_pupil.csv"), index_col=False)
dat_gss = pd.read_csv(Path.joinpath(dir_prep,"all_trials_gss.csv"), index_col=False)

print(list(dat_pupil.columns))

dat_clin = pd.read_excel(Path.joinpath(dir_raw,"behav_data.xlsx"), index_col=False)


['ID', 'Trial n', 'Feedback type', 'Feedback angle', 'Pupil size', 'Percentage bad pupil samples', 'Group']


## Review clinical data

In [26]:
dat_clin.head()

,ID,Alter,Geschlecht,BDI-Score,Schahmann semantische Wortflüssigkeit,Schahmann phonematische Wortflüssigkeit,Schahmann Kategorie-Wechsel,Schahmann Zahlen-spanne vorwärts,Schahmann Zahlen-spanne rückwärts,Schahmann Würfel,...,Tetras Tremor OE kinet. Tremor LI,Tetras Tremor UE,Tetras Archimedes RE,Tetras Archimedes LI,Tetras Handschrift,Tetras Punktan-näherung RE,Tetras Punktan-näherung LI,Tetras Tremor im Stehen,Tetras Gesamtscore Aufgaben-skala,Gesamtscore Tetras
0,p114,35.0,m,4.0,26.0,10.0,12.0,6.0,4.0,15.0,...,1.5,2.0,1.0,2.0,1.0,1.0,1.5,0.0,17.0,33.0
1,p115,46.0,w,2.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.5,1.0,1.0,1.0,1.0,1.5,2.0,0.0,17.5,35.5
2,p116,23.0,m,5.0,26.0,13.0,15.0,5.0,4.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
3,p117,81.0,m,13.0,24.0,7.0,9.0,4.0,2.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0
4,p118,51.0,m,11.0,26.0,17.0,15.0,5.0,5.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0


In [36]:
# clean data
pat = dat_clin[dat_clin["ID"].str.contains("p2|c3")]
print(list(pat.columns))

['ID', 'Alter', 'Geschlecht', 'BDI-Score', 'Schahmann semantische Wortflüssigkeit', 'Schahmann phonematische Wortflüssigkeit', 'Schahmann Kategorie-Wechsel', 'Schahmann Zahlen-spanne vorwärts', 'Schahmann Zahlen-spanne rückwärts', 'Schahmann Würfel ', 'Schahmann verbaler Abruf', 'Schahmann Gemeinsam-keiten', 'Schahmann Go/No-Go', 'Schahmann Affekt', 'Schahmann Gesamtwert \n(x/120)', 'Schahmann Gesamtrwert\n(x/10)', 'Tetras Sprechen', 'Tetras mit dem Löffel essen', 'Tetras aus einem Glas trinken', 'Tetras Körper-hygiene', 'Tetras Ankleiden', 'Tetras Schütten/Ein-schenken', 'Tetras Tablett tragen', 'Tetras Schlüssel benutzen', 'Tetras Schreiben', 'Tetras Arbeiten', 'Tetras Tätigkeit mit schwerster Beeinträchti-gung', 'Tetras Grad der Beeinträchti-gung ', 'Tetras soziale Beeinträchti-gung', 'Tetras Gesamtscore Interview', 'Tetras Kopftremor', 'Tetras Gesichts-tremor', 'Tetras Stimm-tremor', 'Tetras Tremor OE ausgestreckt RE', 'Tetras Tremor OE ausgestreckt LI', 'Tetras Tremor OE Flügelsch